##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Conversiones de espacio de colores

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/colorspace"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/colorspace.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

En visión por computadora, el espacio de colores seleccionado podría tener un impacto significativo en el rendimiento del modelo. Si bien `RGB` es el espacio de colores más común, en muchas situaciones el modelo funciona mejor cuando se cambia a espacios de colores alternativos como `YUV`, `YCbCr`, `XYZ (CIE)`, etc.

El paquete `tensorflow-io` proporciona una lista de API de conversión de espacios de colores que se pueden usar para preparar y aumentar los datos de la imagen.

## Preparación

### Instale los paquetes necesarios y reinicie el tiempo de ejecución

In [ ]:
!pip install tensorflow-io

### Descargue la imagen de muestra

El ejemplo de imagen que se usa en este tutorial es un [gato en la nieve](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg), aunque podría reemplazarse por cualquier imagen JPEG.

Lo siguiente descargará la imagen y la guardará en el disco local como `sample.jpg`:

In [ ]:
!curl -o sample.jpg -L https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg

!ls -ls sample.jpg

## Uso

### Leer archivo de imagen

Lea y decodifique la imagen en un tensor `uint8` de forma `(213, 320, 3)`

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

image = tf.image.decode_jpeg(tf.io.read_file('sample.jpg'))

print(image.shape, image.dtype)

La imagen se puede mostrar con:

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(image)
plt.axis('off')
plt.show()

### Convertir RGB a escala de grises

Una imagen `RGB` se puede convertir a `Grayscale` para reducir el canal de 3 a 1 con `tfio.experimental.color.rgb_to_grayscale`:

In [ ]:
grayscale = tfio.experimental.color.rgb_to_grayscale(image)

print(grayscale.shape, grayscale.dtype)

# use tf.squeeze to remove last channel for plt.imshow to display:
plt.figure()
plt.imshow(tf.squeeze(grayscale, axis=-1), cmap='gray')
plt.axis('off')
plt.show()

### Convertir RGB a BGR

Algunos fabricantes de software de imágenes y cámaras pueden preferir `BGR`, que se puede obtener a través de `tfio.experimental.color.rgb_to_bgr`:

In [ ]:
bgr = tfio.experimental.color.rgb_to_bgr(image)

print(bgr.shape, bgr.dtype)

plt.figure()
plt.imshow(bgr)
plt.axis('off')
plt.show()

### Convertir RGB a CIE XYZ

`CIE XYZ` (o `CIE 1931 XYZ` es un espacio de colores común que se usa en muchos programas de procesamiento de imágenes. La siguiente es la conversión de RGB a `CIE XYZ` a través de `tfio.experimental.color.rgb_to_xyz`. Nota: `tfio.experimental.color.rgb_to_xyz` asume una entrada de punto flotante en el rango de `[0, 1]`, por lo que se necesita un preprocesamiento adicional:

In [ ]:
# convert to float32
image_float32 = tf.cast(image, tf.float32) / 255.0

xyz_float32 = tfio.experimental.color.rgb_to_xyz(image_float32)

# convert back uint8
xyz = tf.cast(xyz_float32 * 255.0, tf.uint8)

print(xyz.shape, xyz.dtype)

plt.figure()
plt.imshow(xyz)
plt.axis('off')
plt.show()

### Convertir RGB a YCbCr

Finalmente, `YCbCr` es el espacio de colores predeterminado en muchos sistemas de vídeo. La conversión a `YCbCr` se puede realizar a través de `tfio.experimental.color.rgb_to_ycbcr`:

In [ ]:
ycbcr = tfio.experimental.color.rgb_to_ycbcr(image)

print(ycbcr.shape, ycbcr.dtype)

plt.figure()
plt.imshow(ycbcr, cmap='gray')
plt.axis('off')
plt.show()

Sin embargo, lo más interesante es que `YCbCr` podría descomponerse en componentes `Y'` (luma), `Cb` (croma de diferencia de azul) y `Cr` (croma de diferencia de rojo), y cada componente contiene información perceptualmente significativa:

In [ ]:
y, cb, cr = ycbcr[:,:,0], ycbcr[:,:,1], ycbcr[:,:,2]

# Y' component
plt.figure()
plt.imshow(y, cmap='gray')
plt.axis('off')
plt.show()

# Cb component
plt.figure()
plt.imshow(cb, cmap='gray')
plt.axis('off')
plt.show()

# Cr component
plt.figure()
plt.imshow(cr, cmap='gray')
plt.axis('off')
plt.show()